# Reference data accuracy assessment by Radiant Earth

Radiant Earth is conducting an accuracy assessment of DE Africa cropmask reference data using the airbus high-res satellite archive. This notebook produces a confusion matrix between DE AFrica's labels and Radiant Earth's labels.  

Inputs will be:

1. `<AEZ-region_RE_sample_validation.geojson>` : The results from collecting training data in the CEO tool

Output will be:
1. A `confusion error matrix` containing Overall, Producer's, and User's accuracy, along with the F1 score.

***

In [1]:
import pandas as pd
import numpy as np
import seaborn as sn
import geopandas as gpd
import matplotlib.pyplot as plt
from sklearn.metrics import f1_score

## Analysis Parameters

In [2]:
folder = 'data/training_validation/collect_earth/eastern/'
gjson =  'data/training_validation/collect_earth/eastern/Eastern_region_RE_sample_validated.geojson'

### Load the dataset

In [3]:
#ground truth shapefile
df = gpd.read_file(gjson)
df.head()

,SMPL_SAMPLEID,Class,Validation_Class,geometry
0,534,non-crop,N/A,"POLYGON ((35.70109 5.63821, 35.70150 5.63821, ..."
1,1691,crop,crop,"POLYGON ((40.19302 6.76901, 40.19344 6.76901, ..."
2,1405,non-crop,non-crop,"POLYGON ((34.68887 7.97203, 34.68928 7.97203, ..."
3,1244,non-crop,non-crop,"POLYGON ((35.65743 10.81843, 35.65784 10.81843..."
4,427,crop,crop,"POLYGON ((39.04498 12.68279, 39.04539 12.68279..."


### Clean up dataframe


In [4]:
# this line if testing sample:
# df = df[['lon', 'lat', 'smpl_class','Is the sample area entirely: crop, non-crop, mixed, or unsure?']]

#This line if entire dataset:
# df = df[['lon', 'lat', 'smpl_sampleid', 'smpl_gfsad_samp','smpl_class','Is the sample area entirely: crop, non-crop, mixed, or unsure?']]

#rename columns
df = df.rename(columns={'Class':'Prediction',
                        'Validation_Class':'Actual'})
df.head()

,SMPL_SAMPLEID,Prediction,Actual,geometry
0,534,non-crop,N/A,"POLYGON ((35.70109 5.63821, 35.70150 5.63821, ..."
1,1691,crop,crop,"POLYGON ((40.19302 6.76901, 40.19344 6.76901, ..."
2,1405,non-crop,non-crop,"POLYGON ((34.68887 7.97203, 34.68928 7.97203, ..."
3,1244,non-crop,non-crop,"POLYGON ((35.65743 10.81843, 35.65784 10.81843..."
4,427,crop,crop,"POLYGON ((39.04498 12.68279, 39.04539 12.68279..."


***

### Reclassify prediction & actual columns

1 = crop, 
0 = non-crop

In [5]:
df['Prediction'] = np.where(df['Prediction']=='non-crop', 0, df['Prediction'])
df['Prediction'] = np.where(df['Prediction']=='crop', 1, df['Prediction'])

df['Actual'] = np.where(df['Actual']=='non-crop', 0, df['Actual'])
df['Actual'] = np.where(df['Actual']=='crop', 1, df['Actual'])

df.head()

,SMPL_SAMPLEID,Prediction,Actual,geometry
0,534,0,N/A,"POLYGON ((35.70109 5.63821, 35.70150 5.63821, ..."
1,1691,1,1,"POLYGON ((40.19302 6.76901, 40.19344 6.76901, ..."
2,1405,0,0,"POLYGON ((34.68887 7.97203, 34.68928 7.97203, ..."
3,1244,0,0,"POLYGON ((35.65743 10.81843, 35.65784 10.81843..."
4,427,1,1,"POLYGON ((39.04498 12.68279, 39.04539 12.68279..."


### Generate a confusion matrix with all classes

In [6]:
confusion_matrix = pd.crosstab(df['Actual'],
                               df['Prediction'],
                               rownames=['Actual'],
                               colnames=['Prediction'],
                               margins=True)

confusion_matrix

Prediction,0,1,All
Actual,,,
0,47,1,48
1,0,40,40
N/A,3,1,4
mixed,0,8,8
All,50,50,100


### Reclassify into a binary assessment

In [21]:
counts = df.groupby('Actual').count()

print("Total number of samples: " + str(len(df)))
print("Number of 'mixed' samples: "+ str(counts[counts.index=='mixed']['Prediction'].values[0]))
print("Number of 'N/A' samples: "+ str(counts[counts.index=='N/A']['Prediction'].values[0]))

print("Dropping 'mixed' and 'N/A' samples")

df = df.drop(df[df['Actual']=='mixed'].index)
df = df.drop(df[df['Actual']=='N/A'].index)

Total number of samples: 100
Number of 'mixed' samples: 8
Number of 'N/A' samples: 4
Dropping 'mixed' and 'N/A' samples


---

### Recreate confusion matrix

In [22]:
confusion_matrix = pd.crosstab(df['Actual'],
                               df['Prediction'],
                               rownames=['Actual'],
                               colnames=['Prediction'],
                               margins=True)

confusion_matrix

Prediction,0,1,All
Actual,,,
0,47,1,48
1,0,40,40
All,47,41,88


### Calculate User's and Producer's Accuracy

`Producer's Accuracy`

In [23]:
confusion_matrix["Producer's"] = [confusion_matrix.loc[0, 0] / confusion_matrix.loc[0, 'All'] * 100,
                              confusion_matrix.loc[1, 1] / confusion_matrix.loc[1, 'All'] * 100,
                              np.nan]

`User's Accuracy`

In [24]:
users_accuracy = pd.Series([confusion_matrix[0][0] / confusion_matrix[0]['All'] * 100,
                                confusion_matrix[1][1] / confusion_matrix[1]['All'] * 100]
                         ).rename("User's")

confusion_matrix = confusion_matrix.append(users_accuracy)

`Overall Accuracy`

In [25]:
confusion_matrix.loc["User's","Producer's"] = (confusion_matrix.loc[0, 0] + 
                                                confusion_matrix.loc[1, 1]) / confusion_matrix.loc['All', 'All'] * 100

`F1 Score`

The F1 score is the harmonic mean of the precision and recall, where an F1 score reaches its best value at 1 (perfect precision and recall), and is calculated as:

$$
\begin{aligned}
\text{Fscore} = 2 \times \frac{\text{UA} \times \text{PA}}{\text{UA} + \text{PA}}.
\end{aligned}
$$

Where UA = Users Accuracy, and PA = Producer's Accuracy

In [26]:
fscore = pd.Series([(2*(confusion_matrix.loc["User's", 0]*confusion_matrix.loc[0, "Producer's"]) / (confusion_matrix.loc["User's", 0]+confusion_matrix.loc[0, "Producer's"])) / 100,
                    f1_score(df['Actual'].astype(np.int8), df['Prediction'].astype(np.int8), average='binary')]
                         ).rename("F-score")

confusion_matrix = confusion_matrix.append(fscore)

### Tidy Confusion Matrix

* Limit decimal places,
* Add readable class names
* Remove non-sensical values 

In [27]:
# round numbers
confusion_matrix = confusion_matrix.round(decimals=2)

In [28]:
# rename booleans to class names
confusion_matrix = confusion_matrix.rename(columns={0:'Non-crop', 1:'Crop', 'All':'Total'},
                                            index={0:'Non-crop', 1:'Crop', 'All':'Total'})

In [29]:
#remove the nonsensical values in the table
confusion_matrix.loc["User's", 'Total'] = '--'
confusion_matrix.loc['Total', "Producer's"] = '--'
confusion_matrix.loc["F-score", 'Total'] = '--'
confusion_matrix.loc["F-score", "Producer's"] = '--'

In [30]:
confusion_matrix

Prediction,Non-crop,Crop,Total,Producer's
Actual,,,,
Non-crop,47.00,1.00,48,97.92
Crop,0.00,40.00,40,100
Total,47.00,41.00,88,--
User's,100.00,97.56,--,98.86
F-score,0.99,0.99,--,--


### Export csv

In [31]:
confusion_matrix.to_csv(folder+ 'radiant_earth_reference_data_accuracy_results.csv')